In [ ]:
import random
import time
from paho.mqtt import client as mqtt_client
import hashlib
import os
import ldap3
from ldap3.core.exceptions import LDAPException

broker="192.168.198.128"
port =1883
topic = "/python/mqtt"
ldap_server = " "
ldap_username = "chamodi"
ldap_password = "London123"

# generate client ID with pub prefix randomly
client_id = f'python-mqtt-{random.randint(0, 1000)}'
username = 'emqx'
password = 'public'
salt = os.urandom(32) # Remember this

hash_password = hashlib.pbkdf2_hmac(
    'sha256', # The hash digest algorithm for HMAC
    password.encode('utf-8'), # Convert the password to bytes
    salt, # Provide the salt
    100000 # It is recommended to use at least 100,000 iterations of SHA-256 
)

# print(hash_password)

# ldap_login Auth
def ldap_auth(username, password):
    try:
        with ldap3.Connection(ldap_server, user=username, password=password) as conn:
            print(conn.result["description"]) # "success" if bind is ok
            return True
    except LDAPException:
        print('Unable to connect to LDAP server')
        return False
    
def connect_mqtt():
    def on_connect(client, userdata, flags, rc):
        if rc == 0:
            print("Connected to MQTT Broker!")
        else:
            print("Failed to connect, return code %d\n", rc)

    client = mqtt_client.Client(client_id)
    client.username_pw_set(username, hash_password)
    client.on_connect = on_connect
    client.connect(broker, port)
    return client


def publish(client):
    msg_count = 0
    while True:
        time.sleep(1)
        msg = f"messages: {msg_count}"
        result = client.publish(topic, msg)
        # result: [0, 1]
        status = result[0]
        if status == 0:
            print(f"Send `{msg}` to topic `{topic}`")
        else:
            print(f"Failed to send message to topic {topic}")
        msg_count += 1


def run():
    client = connect_mqtt()
    client.loop_start()
    ldap_status = ldap_auth(ldap_username, ldap_password)
    if(ldap_status):
        publish(client)
    else:
        print("LDAP server - User name and password do not match");


if __name__ == '__main__':
    run()